# Seq2Seq Prefix-Tuning for SQL

This is the main notebook for prefix-tuning *codet5p-220m-bimodal seq2seq* on SQL programming language dataset.

## Clone the repository

In [1]:
!git clone https://github.com/leiluk1/CodeSearcher.git && cd CodeSearcher/ && git checkout dev/embeddings

Cloning into 'CodeSearcher'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 252 (delta 138), reused 188 (delta 83), pack-reused 0bjects:  28% (72/252), 2.35 MiB | 923.00 KiB/sReceiving objects:  30% (78/252), 4.40 MiB | 1.07 MiB/s  Receiving objects:  31% (79/252), 5.85 MiB | 1.21 MiB/sReceiving objects:  33% (84/252), 5.85 MiB | 1.21 MiB/sReceiving objects:  35% (89/252), 5.85 MiB | 1.21 MiB/sReceiving objects:  42% (106/252), 5.85 MiB | 1.21 MiB/sReceiving objects:  90% (227/252), 6.58 MiB | 1.30 MiB/sReceiving objects:  92% (232/252), 6.58 MiB | 1.30 MiB/s
Receiving objects: 100% (252/252), 7.17 MiB | 1.20 MiB/s, done.
Resolving deltas: 100% (138/138), done.
branch 'dev/embeddings' set up to track 'origin/dev/embeddings'.
Switched to a new branch 'dev/embeddings'


## Set up the required dependencies

In [2]:
!pip install dataprep gdown py7zr transformers peft evaluate rouge_score fire loguru --quiet

In [3]:
# import wandb
# wandb.login(key='')

wandb: Currently logged in as: ley-khaertdinova. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/buiniy_yarik/.netrc


True

In [4]:
!mkdir -p CodeSearcher/output
!mkdir -p CodeSearcher/data/raw
!gdown 1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
!unzip -q -d CodeSearcher/data/raw ./XLCoST_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
From (redirected): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6&confirm=t&uuid=a4c3abac-d04d-4ddf-8a14-40f56eda54f7
To: /home/buiniy_yarik/DataspellProjects/Karina_PMLDL/XLCoST_data.zip
100%|████████████████████████████████████████| 298M/298M [00:39<00:00, 7.61MB/s]


## Perform prefix-tuning 

For more details, please check the code in `src/models/train.py` in our repository.

In [5]:
!export DATASETS_VERBOSITY=error
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/train.py seq2seq prefix \
    --output_dir="./output" \
    --epochs=10 \
    --num_virtual_tokens=10 \
    --language="SQL" 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2023-11-23 12:16:07.769 | INFO     | __main__:_setup_peft_model:28 - trainable params: 184,320 || all params: 223,267,074 || trainable%: 0.08255583624480159
2023-11-23 12:16:13.083 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-23 12:16:13.105 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL train dataset length: 62930
2023-11-23 12:16:17.800 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-23 12:16:17.809 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL val dataset length: 6993
2023-11-23 12:16:22.650 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-23 12:16:22.660 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL test dataset length: 7770
Tokenizing dataset: 100%|█████████| 7770/7770 [00

## Save checkpoint

In [7]:
!zip -r seq2seq_prefix_sql.zip CodeSearcher/output

  adding: CodeSearcher/output/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-3500/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-3500/README.md (deflated 65%)
  adding: CodeSearcher/output/checkpoint-3500/adapter_model.safetensors (deflated 7%)
  adding: CodeSearcher/output/checkpoint-3500/adapter_config.json (deflated 37%)
  adding: CodeSearcher/output/checkpoint-3500/tokenizer_config.json (deflated 95%)
  adding: CodeSearcher/output/checkpoint-3500/special_tokens_map.json (deflated 82%)
  adding: CodeSearcher/output/checkpoint-3500/added_tokens.json (deflated 37%)
  adding: CodeSearcher/output/checkpoint-3500/vocab.json (deflated 59%)
  adding: CodeSearcher/output/checkpoint-3500/merges.txt (deflated 54%)
  adding: CodeSearcher/output/checkpoint-3500/tokenizer.json (deflated 72%)
  adding: CodeSearcher/output/checkpoint-3500/training_args.bin (deflated 48%)
  adding: CodeSearcher/output/checkpoint-3500/optimizer.pt (deflated 53%)
  adding: CodeSearcher/output/checkpoint

## Evaluation

In this step, test the model using *Mean Reciprocal Rank (MRR)* as the evaluation metric. 

For more details, please refer to `src/models/evaluation.py` in the repository.

In [6]:
!export DATASETS_VERBOSITY=error
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/evaluation.py \
    --tuned_ckpt_path="output/best_ckpt" \
    --num_virtual_tokens=10 \
    --language="SQL"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2023-11-23 14:04:42.132 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-23 14:04:42.170 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL train dataset length: 62930
2023-11-23 14:04:47.007 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-23 14:04:47.019 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL val dataset length: 6993
2023-11-23 14:04:52.055 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-23 14:04:52.065 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL test dataset length: 7770
Testing the model: 100%|██████████████████████| 486/486 [00:18<00:00, 26.83it/s]
2023-11-23 14:05:14.540 | INFO     | __main__:evaluation:34 - Test MRR for SQL = 0.022741608300840738


# Results:

- Test MRR for SQL: **0.02274161**;
- Epochs: **10**;
- Trainable params: 184,320 || all params: 223,267,074 || trainable%: 0.08255583624480159.

You can also check the validation plots in [wanb report](https://api.wandb.ai/links/ley-khaertdinova/13lvn64p).